<a href="https://colab.research.google.com/github/agalvezm/ACE2_scRNAseq/blob/master/tcc/GSE120508_GSM3052919_from_gc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSE120508_GSM3052919

This notebook uses the filtered count matrix resulting from the following google colab notebook. https://github.com/agalvezm/ACE2_scRNAseq/blob/master/notebooks_countmatrices/GSE120508_GSM3052919.ipynb Please run the notebook above and upload the results file in /content

In [1]:
# define the values for the analysis

# accession id for the data
id = "GSE120508"
samp_id = ["GSM3052919"]

# If only bam available files, set bam = True, Fill link and filename
bam = True

# If fastq links available but are not ffq links
fastq_ffqlinks = True


if bam:
  # Assign link to python variable
  link_to_bam = ["https://sra-pub-src-1.s3.amazonaws.com/SRR6860521/14737X1.bam.1",	"https://sra-pub-src-1.s3.amazonaws.com/SRR6860522/14737X2.bam.1"]
  # Assigns the link to the bash variable BAM_LINK. To be used by wget
  %env BAM_LINK=$link_to_bam

  # Assign filename to python variable. Used to convert to fastq and remove bam file to fastq after conversion

  bam_filename=["14737X1.bam.1","14737X2.bam.1"]


if not bam:

  # Copy and paste the links from the ACE2 scRNAseq datasets google  spreadsheet

  links_raw = ""

  # Convert it to a list where each link is an element
  
  fastqs = links_raw.split()


env: BAM_LINK=['https://sra-pub-src-1.s3.amazonaws.com/SRR6860521/14737X1.bam.1', 'https://sra-pub-src-1.s3.amazonaws.com/SRR6860522/14737X2.bam.1']


In [2]:
no_samples = 1

fastqs_per_sample = [4, 8, 8] 

sample_id = samp_id

database_id = [id] * no_samples

tissue = ["testis"] * no_samples

cell_type = ["total"] * no_samples

condition = ["deceased donor 2, 24 years old"]

species = ["human"] * no_samples

technology = ["10xv2"] * no_samples

paper = ["Sungnak et al 2020"] * no_samples

figure = ["Fig 1"] * no_samples

# Set string variables for kb functions

species_kb = species[0]

technology_kb = technology[0]



# Imports and installs

In [3]:
# install and import necessary software

# Install kb and scanpy
!pip -q install kb-python 
!pip -q install scanpy

import re
import os

# Setup

import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import scanpy as sc
from scipy import stats

from collections import OrderedDict
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.preprocessing import scale

from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from matplotlib import cm
from matplotlib.lines import Line2D

def nd(arr):
    return np.asarray(arr).reshape(-1)
def yex(ax):
    lims = [np.min([ax.get_xlim(), ax.get_ylim()]),
            np.max([ax.get_xlim(), ax.get_ylim()])]

    # now plot both limits against eachother
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    return ax

def trim_axs(axs, N):
    """little helper to massage the axs list to have correct length..."""
    axs = axs.flat
    for ax in axs[N:]:
        ax.remove()
    return axs[:N]

import warnings
warnings.filterwarnings('ignore')

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

     |████████████████████████████████| 59.1MB 96kB/s 
     |████████████████████████████████| 133kB 47.5MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 13.2MB 41.0MB/s 
     |████████████████████████████████| 10.3MB 18.9MB/s 
     |████████████████████████████████| 112kB 41.3MB/s 
     |████████████████████████████████| 81kB 2.9MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
     |████████████████████████████████| 1.2MB 40.0MB/s 
     |████████████████████████████████| 51kB 4.9MB/s 


# Downloads: (bam (if bam) and index

In [4]:
if bam:

  # Install bamtofastq from 10x website (only bam files available)
  !wget http://cf.10xgenomics.com/misc/bamtofastq-1.2.0
  !chmod +x bamtofastq-1.2.0
  # Download the bam file
  for link in link_to_bam:
    !wget -- continue $link




--2021-04-13 18:33:54--  http://cf.10xgenomics.com/misc/bamtofastq-1.2.0
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 104.18.1.173, 104.18.0.173, 2606:4700::6812:1ad, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.1.173|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cf.10xgenomics.com/misc/bamtofastq-1.2.0 [following]
--2021-04-13 18:33:54--  https://cf.10xgenomics.com/misc/bamtofastq-1.2.0
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|104.18.1.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13288280 (13M) [binary/octet-stream]
Saving to: ‘bamtofastq-1.2.0’

bamtofastq-1.2.0    100%[===================>]  12.67M  49.9MB/s    in 0.3s    

2021-04-13 18:33:55 (49.9 MB/s) - ‘bamtofastq-1.2.0’ saved [13288280/13288280]

--2021-04-13 18:33:55--  http://continue/
Resolving continue (continue)... failed: Name or service not known.
wget: unable to resolve host address ‘continue’
-

In [5]:
if bam:
  # Convert to fastq
  for filename in bam_filename:
    !./bamtofastq-1.2.0 --reads-per-fastq=500000000 $filename ./fastqs$filename

  # Remove original bam file to save space
  for filename in bam_filename:
    !rm $filename


bamtofastq v1.2.0
Args { arg_bam: "14737X1.bam.1", arg_output_path: "./fastqs14737X1.bam.1", flag_nthreads: 4, flag_locus: None, flag_bx_list: None, flag_reads_per_fastq: 500000000, flag_gemcode: false, flag_lr20: false, flag_cr11: false }
Writing finished.  Observed 263055092 read pairs. Wrote 263055092 read pairs
bamtofastq v1.2.0
Args { arg_bam: "14737X2.bam.1", arg_output_path: "./fastqs14737X2.bam.1", flag_nthreads: 4, flag_locus: None, flag_bx_list: None, flag_reads_per_fastq: 500000000, flag_gemcode: false, flag_lr20: false, flag_cr11: false }
Writing finished.  Observed 288683878 read pairs. Wrote 288683878 read pairs


In [8]:
!for file in /content/fastqs/14737X1_mRNA_MissingLibrary_1_CBVUMANXX_/*; do mv "$file" "$(basename "$file")_"; done;

In [9]:
!ls

14737X1_mRNA_MissingLibrary_1_CBVUMANXX_  bamtofastq_S1_L007_R1_001.fastq.gz_
bamtofastq-1.2.0			  bamtofastq_S1_L007_R2_001.fastq.gz_
bamtofastq_S1_L005_I1_001.fastq.gz_	  bamtofastq_S1_L008_I1_001.fastq.gz_
bamtofastq_S1_L005_R1_001.fastq.gz_	  bamtofastq_S1_L008_R1_001.fastq.gz_
bamtofastq_S1_L005_R2_001.fastq.gz_	  bamtofastq_S1_L008_R2_001.fastq.gz_
bamtofastq_S1_L006_I1_001.fastq.gz_	  fastqs14737X1.bam.1
bamtofastq_S1_L006_R1_001.fastq.gz_	  fastqs14737X2.bam.1
bamtofastq_S1_L006_R2_001.fastq.gz_	  sample_data
bamtofastq_S1_L007_I1_001.fastq.gz_


In [10]:
!mv *.gz_ /content/fastqs14737X2.bam.1/14737X2_mRNA_MissingLibrary_1_CBVUMANXX

In [11]:
# Store fastq names on a list

if bam:
  # cd into fastqs folder
  %cd /content/fastqs$filename

  #store the name of the folder generated by bamtofastq
  _filename = os.listdir()[0]

  # cd into that folder
  %cd $_filename

  # store fastq names in a list
  fastqs = os.listdir()


  # Remove I1 and R3 reads not relevant for our analysis

  # Initialize list containing elements to remove
  remov_elem = []

  print ("\n\nThis is the complete list of fastqs:\n -----------")
  for elem in fastqs:
    print (elem)

  # Search index (I1 or R3) fastqs and remove them from list
  for elem in fastqs:
    if re.search("_R3_", elem) or re.search("_I1_", elem):
      remov_elem = remov_elem +[elem]

  fastqs = [elem for elem in fastqs if elem not in remov_elem] 

  print ("\n\nThis is the filtered list of fastqs:\n -----------")
  for elem in fastqs:
    print (elem)


/content/fastqs14737X2.bam.1
/content/fastqs14737X2.bam.1/14737X2_mRNA_MissingLibrary_1_CBVUMANXX


This is the complete list of fastqs:
 -----------
bamtofastq_S1_L006_R2_001.fastq.gz
bamtofastq_S1_L007_I1_001.fastq.gz_
bamtofastq_S1_L005_R2_001.fastq.gz_
bamtofastq_S1_L007_R1_001.fastq.gz_
bamtofastq_S1_L006_R1_001.fastq.gz_
bamtofastq_S1_L006_R1_001.fastq.gz
bamtofastq_S1_L008_R2_001.fastq.gz
bamtofastq_S1_L007_R2_001.fastq.gz_
bamtofastq_S1_L008_R1_001.fastq.gz_
bamtofastq_S1_L005_R1_001.fastq.gz_
bamtofastq_S1_L005_R1_001.fastq.gz
bamtofastq_S1_L006_I1_001.fastq.gz
bamtofastq_S1_L007_R2_001.fastq.gz
bamtofastq_S1_L006_R2_001.fastq.gz_
bamtofastq_S1_L008_I1_001.fastq.gz_
bamtofastq_S1_L006_I1_001.fastq.gz_
bamtofastq_S1_L008_R1_001.fastq.gz
bamtofastq_S1_L008_I1_001.fastq.gz
bamtofastq_S1_L008_R2_001.fastq.gz_
bamtofastq_S1_L007_R1_001.fastq.gz
bamtofastq_S1_L005_I1_001.fastq.gz
bamtofastq_S1_L005_I1_001.fastq.gz_
bamtofastq_S1_L007_I1_001.fastq.gz
bamtofastq_S1_L005_R2_001.fastq.g

In [12]:
# Remove fastqs that wont be analyzed to save space
if bam:
  for elem in remov_elem:
    !rm $elem

In [13]:
if bam:
  # sort fastqs alphabetically to get R1 and R2 in order
  fastqs = sorted(fastqs)

In [14]:
# wget fastqs from non ffq links in fastqs folder
if not bam and not fastq_ffqlinks:
  !mkdir fastqs
  %cd fastqs
  for link in fastqs:
    !wget $link --continue

  # update fastqs variable with name of files
  fastqs = sorted(os.listdir())

In [15]:
if bam:
  # Download the corresponding Kallisto index to folder containing fastqs
  !kb ref -d $species_kb -i index.idx -g t2g.txt -f1 transcriptome.fasta

if not bam and fastq_ffqlinks:
  %cd /content

  # Download the corresponding Kallisto index to content folder
  !kb ref -d $species_kb -i index.idx -g t2g.txt -f1 transcriptome.fasta

if not bam and not fastq_ffqlinks:
  %cd /content/fastqs

  # Download the corresponding Kallisto index to fastq folder
  !kb ref -d $species_kb -i index.idx -g t2g.txt -f1 transcriptome.fasta

[2021-04-13 21:15:02,242]    INFO Downloading files for human from https://caltech.box.com/shared/static/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz to tmp/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz
100% 2.23G/2.23G [01:47<00:00, 22.2MB/s]
[2021-04-13 21:16:51,086]    INFO Extracting files from tmp/v1nm7lpnqz5syh8dyzdk2zs8bglncfib.gz


In [16]:
# Check to make sure the metadata is in the right order after sorting
print(fastqs)

['bamtofastq_S1_L005_R1_001.fastq.gz', 'bamtofastq_S1_L005_R1_001.fastq.gz_', 'bamtofastq_S1_L005_R2_001.fastq.gz', 'bamtofastq_S1_L005_R2_001.fastq.gz_', 'bamtofastq_S1_L006_R1_001.fastq.gz', 'bamtofastq_S1_L006_R1_001.fastq.gz_', 'bamtofastq_S1_L006_R2_001.fastq.gz', 'bamtofastq_S1_L006_R2_001.fastq.gz_', 'bamtofastq_S1_L007_R1_001.fastq.gz', 'bamtofastq_S1_L007_R1_001.fastq.gz_', 'bamtofastq_S1_L007_R2_001.fastq.gz', 'bamtofastq_S1_L007_R2_001.fastq.gz_', 'bamtofastq_S1_L008_R1_001.fastq.gz', 'bamtofastq_S1_L008_R1_001.fastq.gz_', 'bamtofastq_S1_L008_R2_001.fastq.gz', 'bamtofastq_S1_L008_R2_001.fastq.gz_']


# Process fastq files (modify kb command according to fastqs list)


In [26]:
np.sort(fastqs)

array(['bamtofastq_S1_L005_R1_001.fastq.gz',
       'bamtofastq_S1_L005_R1_001.fastq.gz_',
       'bamtofastq_S1_L005_R2_001.fastq.gz',
       'bamtofastq_S1_L005_R2_001.fastq.gz_',
       'bamtofastq_S1_L006_R1_001.fastq.gz',
       'bamtofastq_S1_L006_R1_001.fastq.gz_',
       'bamtofastq_S1_L006_R2_001.fastq.gz',
       'bamtofastq_S1_L006_R2_001.fastq.gz_',
       'bamtofastq_S1_L007_R1_001.fastq.gz',
       'bamtofastq_S1_L007_R1_001.fastq.gz_',
       'bamtofastq_S1_L007_R2_001.fastq.gz',
       'bamtofastq_S1_L007_R2_001.fastq.gz_',
       'bamtofastq_S1_L008_R1_001.fastq.gz',
       'bamtofastq_S1_L008_R1_001.fastq.gz_',
       'bamtofastq_S1_L008_R2_001.fastq.gz',
       'bamtofastq_S1_L008_R2_001.fastq.gz_'], dtype='<U35')

In [27]:
fastqs = ['bamtofastq_S1_L005_R1_001.fastq.gz',
       'bamtofastq_S1_L005_R2_001.fastq.gz',
       'bamtofastq_S1_L005_R1_001.fastq.gz_',
       'bamtofastq_S1_L005_R2_001.fastq.gz_',
       'bamtofastq_S1_L006_R1_001.fastq.gz',
       'bamtofastq_S1_L006_R2_001.fastq.gz',
       'bamtofastq_S1_L006_R1_001.fastq.gz_',
       'bamtofastq_S1_L006_R2_001.fastq.gz_',
       'bamtofastq_S1_L007_R1_001.fastq.gz',
       'bamtofastq_S1_L007_R2_001.fastq.gz',
       'bamtofastq_S1_L007_R1_001.fastq.gz_',
       'bamtofastq_S1_L007_R2_001.fastq.gz_',
       'bamtofastq_S1_L008_R1_001.fastq.gz',
       'bamtofastq_S1_L008_R2_001.fastq.gz',
       'bamtofastq_S1_L008_R1_001.fastq.gz_',
       'bamtofastq_S1_L008_R2_001.fastq.gz_']

In [ ]:
if no_samples == 1:

  # Write the kb count command as a string with all fastqs of the list as an input
  cmd = "kb count --h5ad -i index.idx -g t2g.txt -x " + technology_kb + " -o tccoutput" + sample_id[0] + " "\
  + "--filter bustools --tcc -t 2 --overwrite " + "'" +  "' '".join(fastqs) + "'"
  
  # Execute it
  !$cmd

# If more than one sample, iterate through fastqs accordingly
else:

  # Initializa counter for fastq files
  j = 0

  # Loop over samples for analysis
  for i in range(no_samples):

    fastqs_to_analyze = fastqs[j:j + fastqs_per_sample[i]]
    # Write the kb count command as a string
    cmd = "kb count --h5ad -i ../index.idx -g ../t2g.txt -x " + technology_kb + " -o tccoutput" + sample_id[i] + " \
    --filter bustools --tcc -t 2 --overwrite " + "'" +  "' '".join(fastqs_to_analyze) + "'"

    # Execute it
    !$cmd

    # Update j to move to the next set of fastq
    j = j + fastqs_per_sample[i]






results = {}
tcc_results = {}
for sample in sample_id:
  # Read the filtered gene count matrix
  results["data_" + sample] = anndata.read("/content/result" + sample)


for sample in sample_id:
  output = "bus_raw_" + sample + "_from_gc"
  !mkdir $output
  folder_tcc = "tccoutput" + sample

  # read tcc matrix
  tcc_results[sample] = anndata.read(folder_tcc + "/counts_unfiltered/adata.h5ad")
  # apply gene count matrix's filter
  tcc_results[sample] = tcc_results[sample][results["data_" + sample].obs.index.values]
  # transfer obs data
  tcc_results[sample].obs = results["data_" + sample].obs
  # transfer metadata
  tcc_results[sample].uns = results["data_" + sample].uns
  # write tcc matrix
  tcc_results[sample].write("/content/tcc_" + sample)
  #gzip tcc matrix
  cmd = "gzip /content/tcc_" + sample
  !$cmd
  #gzip unfiltered bus file
  cmd = "gzip " + folder_tcc + "/output.unfiltered.bus"
  !$cmd
  # move and re-name bus file with sample id
  cmd = "mv " + folder_tcc + "/output.unfiltered.bus.gz " + output + "/" + sample +".unfiltered.bus.gz"
  !$cmd
  # gzip all unfiltered counts
  cmd = "gzip " + folder_tcc + "/counts_unfiltered/*"
  !$cmd

  # move unfiltered counts for both
  cmd = "mv " + folder_tcc + "/counts_unfiltered " + output + "/tcc_unfiltered"
  !$cmd

  cmd = "mv " + folder_tcc + "/*.json " + output 
  !$cmd
  # zip all files
  cmd = "zip -r "+ output + ".zip " + output
  !$cmd


import time
time.sleep(60000000)

[2021-04-14 01:50:49,904]    INFO Using index index.idx to generate BUS file to tccoutputGSM3052919 from
[2021-04-14 01:50:49,905]    INFO         bamtofastq_S1_L005_R1_001.fastq.gz
[2021-04-14 01:50:49,905]    INFO         bamtofastq_S1_L005_R2_001.fastq.gz
[2021-04-14 01:50:49,905]    INFO         bamtofastq_S1_L005_R1_001.fastq.gz_
[2021-04-14 01:50:49,905]    INFO         bamtofastq_S1_L005_R2_001.fastq.gz_
[2021-04-14 01:50:49,905]    INFO         bamtofastq_S1_L006_R1_001.fastq.gz
[2021-04-14 01:50:49,905]    INFO         bamtofastq_S1_L006_R2_001.fastq.gz
[2021-04-14 01:50:49,905]    INFO         bamtofastq_S1_L006_R1_001.fastq.gz_
[2021-04-14 01:50:49,905]    INFO         bamtofastq_S1_L006_R2_001.fastq.gz_
[2021-04-14 01:50:49,905]    INFO         bamtofastq_S1_L007_R1_001.fastq.gz
[2021-04-14 01:50:49,905]    INFO         bamtofastq_S1_L007_R2_001.fastq.gz
[2021-04-14 01:50:49,905]    INFO         bamtofastq_S1_L007_R1_001.fastq.gz_
[2021-04-14 01:50:49,905]    INFO         b

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


  adding: bus_raw_GSM3052919_from_gc/ (stored 0%)
  adding: bus_raw_GSM3052919_from_gc/run_info.json (deflated 66%)
  adding: bus_raw_GSM3052919_from_gc/GSM3052919.unfiltered.bus.gz (deflated 4%)
  adding: bus_raw_GSM3052919_from_gc/inspect.json (deflated 57%)
  adding: bus_raw_GSM3052919_from_gc/tcc_unfiltered/ (stored 0%)
  adding: bus_raw_GSM3052919_from_gc/tcc_unfiltered/cells_x_tcc.mtx.gz (deflated 0%)
  adding: bus_raw_GSM3052919_from_gc/tcc_unfiltered/cells_x_tcc.barcodes.txt.gz (deflated 1%)
  adding: bus_raw_GSM3052919_from_gc/tcc_unfiltered/cells_x_tcc.ec.txt.gz (deflated 1%)
  adding: bus_raw_GSM3052919_from_gc/tcc_unfiltered/adata.h5ad.gz (deflated 5%)
  adding: bus_raw_GSM3052919_from_gc/kb_info.json (deflated 79%)
